In [6]:
import langchain

In [7]:
import os
from dotenv import load_dotenv
load_dotenv() 

True

In [10]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

### Example 1 : Simple LLM call with streaming output


In [ ]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage , SystemMessage

In [13]:
model = init_chat_model("groq:llama-3.1-8b-instant")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x11be00ec0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x11be017f0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [15]:
### Create messages
messages = [
      SystemMessage("You are a helpful AI assistant"),
      HumanMessage("What are the top three benefits of playing sports?")
]


In [17]:
### invoking the model 
response = model.invoke(messages)
print(response.content)

Playing sports can have numerous benefits for individuals of all ages. Here are the top three benefits of playing sports:

1. **Physical Health Benefits**: Engaging in sports can improve overall physical health by increasing cardiovascular fitness, strength, flexibility, and coordination. Regular physical activity can also help maintain a healthy weight, reduce the risk of chronic diseases like diabetes and heart disease, and improve sleep quality.

2. **Mental Health Benefits**: Playing sports can have a positive impact on mental health by reducing stress and anxiety levels. The physical activity and social interaction involved in sports can release endorphins, also known as "feel-good" hormones, which can improve mood and overall mental well-being. Additionally, participating in sports can help build confidence, self-esteem, and resilience.

3. **Social Benefits**: Playing sports provides opportunities for socialization and building relationships with teammates, coaches, and opponent

In [21]:
### Streaming example
for chunk in model.stream(messages):
    print(chunk.content , end="" ,flush=True)

Playing sports can have numerous benefits for individuals, both physically and mentally. Here are the top three benefits of playing sports:

1. **Improved Physical Health**: Engaging in sports can significantly improve cardiovascular health, increase muscle strength and endurance, and enhance flexibility. Regular physical activity can also help maintain a healthy weight, reduce the risk of chronic diseases, and improve overall physical fitness.

2. **Mental Well-being and Stress Relief**: Playing sports can have a positive impact on mental health by reducing stress, anxiety, and depression. The physical activity and social interaction involved in sports can release endorphins, also known as "feel-good" hormones, which can improve mood and overall mental well-being.

3. **Development of Life Skills and Social Benefits**: Participating in sports can help individuals develop essential life skills such as teamwork, communication, discipline, and problem-solving. Playing sports also provide

### Dynamic Prompt Templates

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

In [34]:
## Sytem message - sys instruction


## translation app
translation_template = ChatPromptTemplate.from_messages([
    ("system","You are a professional in speaking french and english. Your job is to help user translate the following text {text}from {source_language} to {target_language} language. Maintain the tone and style" ),
    ("user","{text}")
])

## using the template
prompt = translation_template.invoke({
    "source_language":"English",
    "target_language":"Hindi",
    "text":"What is your name?"
})


In [35]:
translated_reponse = model.invoke(prompt)
print(translated_reponse.content)


Aapka naam kya hai? 

(Note: Hindi translation of "What is your name?" is "Aapka naam kya hai?" which is a polite way of asking someone's name in Hindi.)


### Building Our First Chain

In [59]:
from langchain_core.output_parsers import StrOutputParser 
from langchain_core.runnables import RunnableLambda , RunnablePassthrough

def create_story_chain():
    story_prompt= ChatPromptTemplate.from_messages(
        [
            ("system","You are a creative story teller. Write a short and an engaging story based on a give theme , character , character2 and settings. Only make use of the two characters mentioned"),
            ("user","Theme: {theme} \n Main Character:{character} \n Support Character:{character2} \n Setting:{setting}")
        ]
    )

    ## Template for story analysis 
    analysis_prompt = ChatPromptTemplate.from_messages(
        [
                ("system","You are a literary critic. Analyze the following story and provide insights"),
                ("user","{story}")
        ]
    )

    story_chain = (
        story_prompt| model| StrOutputParser()
    )

    def analyze_story(story_text):
        return {"story":story_text}
    
    analysis_chain =(
        story_chain
        | RunnableLambda(analyze_story)
        |analysis_prompt
        |model
        |StrOutputParser()
    )

    return analysis_chain

In [60]:
chain = create_story_chain()
chain


ChatPromptTemplate(input_variables=['character', 'character2', 'setting', 'theme'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a creative story teller. Write a short and an engaging story based on a give theme , character , character2 and settings. Only make use of the two characters mentioned'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['character', 'character2', 'setting', 'theme'], input_types={}, partial_variables={}, template='Theme: {theme} \n Main Character:{character} \n Support Character:{character2} \n Setting:{setting}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x11be00ec0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x11be017f0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('*********

In [62]:
result = chain.invoke({
    "theme":"Romantic",
    "character":"Ayan",
    "character2":"Anjan",
    "setting":"Spring season in Paris"
})

print("Story and analysis: ")
print(result)

Story and analysis: 
What a delightful tale of love and serendipity! As a literary critic, I shall dissect this narrative and offer insights into its various components.

**Imagery and Setting:** The story is set in the City of Love, Paris, during the first day of spring. The vivid descriptions of the Eiffel Tower, the Seine River, and the Luxembourg Gardens evoke a sense of beauty and romance. The author's use of sensory details, such as the warm glow of the sun, the scent of blooming flowers, and the sound of birds chirping, effectively transports the reader to the enchanting setting.

**Character Development:** Ayan, the protagonist, is a young artist with a creative and romantic soul. His initial hesitation to share the bench with Anjan, only to be won over by her charm, reveals his introverted and sentimental nature. Anjan, on the other hand, is a beautiful and confident French woman with a passion for art. Her kindness and genuine interest in Ayan make her a likable and relatable